# Re test mediawiki_edit_attempt
Following up on the task Instrumentation Data-QA for event.mediawiki_edit_attempt [T320281](https://phabricator.wikimedia.org/T320281).   
- The first phase of this testing was conducted in Nov 2022. Several issues were highlighted and discussed. I re tested the data in January 2023 and again in June 2023.  

Status Update: 
On Desktop    
- server side wikitext editor (including Save and Init) events are now being logged!!   
- All client side (ready, loaded, firstchange, abort) events are logged 

On Mobile
- some wiki editor Mobile events were being logged using `wt` and `mf` mediawiki extensions i.e. the same events were being logged twice and double counted. We had reported this in January 2023 to @DLynch/@phuedx  and this [patch](https://gerrit.wikimedia.org/r/c/mediawiki/extensions/WikiEditor/+/895337/) was merged in March 2023, and later reverted in Apr (4/14/23) when the Editing team did big centralizations of editattemptstep logging. 
From @DLynch: the reason we removed that patch was that we thought we’d stopped WikiEditor and the MobileFrontend editor from appearing on the page at the same time with this [patch](https://gerrit.wikimedia.org/r/c/mediawiki/extensions/MobileFrontend/+/904565) for [T334263](https://phabricator.wikimedia.org/T334263). This suggests that we didn’t catch every case that could cause that.


After discussing with @DLynch and @cjming, @DLynch made https://gerrit.wikimedia.org/r/c/mediawiki/extensions/WikiEditor/+/928926 to reapply the original patch which was merged and deployed Jun 15, 2023. As of Jun 20, we have not retested to see if the revert was successful in removing duplicate events. 

## Action items from QA test
- retest Mobile events on wikitext editor and check any discrepancies between mobile and desktop events on MEA and EAS tables. If they don't match, dig deeper and talk with Metrics Platform team and make aware to the implementation team in Product. 

In [1]:
import wmfdata as wmf

### Compare between editattemptstep and mediawiki_edit_attempt
#### 1. wikitext events  
We will start with wikitext as all previous attempts to QA wikitext editor events failed since Metrics Platform was not programmed for it.  

In [3]:
wt_eas_query=wmf.spark.run('''
SELECT 
  DISTINCT event.action AS action, 
  event.platform AS platform,
  count(1) AS wikitext_events
  FROM event.editattemptstep
  WHERE event.editor_interface = 'wikitext'
  AND year = 2023
  AND month= 6
  AND wiki = 'testwiki' 
  GROUP BY action, platform
''')

SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/07 18:12:32 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/06/07 18:12:32 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
23/06/07 18:12:32 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
23/06/07 18:12:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/06/07 18:12:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/06/07 18:12:42 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 13000. Attempting port 13001.
23/06/07 18:12:42 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on 

In [4]:
wt_eas_query.sort_values(by=['wikitext_events','action'],ascending=False)

,action,platform,wikitext_events
11,ready,desktop,92
5,loaded,desktop,91
10,firstChange,desktop,43
6,abort,desktop,31
8,ready,phone,13
4,loaded,phone,13
7,init,phone,13
0,firstChange,phone,7
12,abort,phone,6
9,saveIntent,phone,3


In [9]:
wt_mea_query= wmf.spark.run('''
SELECT 
  DISTINCT name, 
  count(1) AS wikitext_events
  FROM event.mediawiki_edit_attempt
  WHERE custom_data["editor_interface"].value = "wikitext"
  AND year = 2023
  AND month= 6
  AND mediawiki.database = "testwiki"
  GROUP BY name
''')

In [10]:
wt_mea_query.sort_values(by=['wikitext_events','name'],ascending=False)

,name,wikitext_events
3,eas.wt.ready,93
0,eas.wt.loaded,92
6,eas.wt.first_change,43
7,eas.wt.abort,33
2,eas.mf.ready,14
12,eas.mf.loaded,14
4,eas.mf.init,13
9,eas.mf.first_change,7
11,eas.mf.abort,6
5,eas.mf.save_intent,3


In [11]:
#Events in mediawiki_edit_attempt
wt_mea_query.wikitext_events.sum()

324

In [29]:
#Events in editattemptstep
wt_eas_query.wikitext_events.sum()

318

The total number of wikitext events logged in editattemptstep and mediawiki_edit_attempt are close. This means the [patch](https://gerrit.wikimedia.org/r/c/mediawiki/extensions/WikiEditor/+/902441/) submitted for WikiEditor extensions is working correctly and events are logging successfully in Metrics Platform.  

In the Metrics Platform world, 'mf' is mobile frontend which means events from mobile device. And the events in mediawiki_edit_attempt are nearly the same as the number of events for each action in editattemptstep for Mobile.  
We observe this for desktop i.e. 'wt' events in mediawiki_edit_attempt. 

There is a slight variation in ready and loaded action on mediawiki_edit_attempt and this could be related to wikieditor JS and the mobile JS both running at the same time (comment https://phabricator.wikimedia.org/T320281#8664796 ) 
A mobile event is getting logged as desktop event or both. However the proportion of such events is miniscule

### 2. VE events

In [12]:
ve_eas_query=wmf.spark.run('''
SELECT 
  DISTINCT event.action AS action, 
  count(1) AS ve_events
  FROM event.editattemptstep
  WHERE event.editor_interface = 'visualeditor'
  AND year = 2023
  AND month= 6
  AND wiki = 'testwiki' 
  GROUP BY action
''')

In [13]:
ve_eas_query.ve_events.sum()

309

In [14]:
ve_mea_query= wmf.spark.run('''
SELECT 
  DISTINCT name, 
  count(1) AS ve_events
  FROM event.mediawiki_edit_attempt
  WHERE custom_data["editor_interface"].value = "visualeditor"
  AND year = 2023
  AND month= 6
  AND mediawiki.database = "testwiki"
  GROUP BY name
''')


In [15]:
ve_mea_query.ve_events.sum()

309

In [14]:
ve_eas_query.sort_values(by=['ve_events','action'],ascending=False)

,action,ve_events
6,init,59
7,ready,54
5,loaded,54
4,firstChange,33
2,saveIntent,27
3,saveSuccess,26
0,saveAttempt,26
1,abort,25


In [15]:
ve_mea_query.sort_values(by=['ve_events','name'],ascending=False)

,name,ve_events
17,eas.ve.init,48
21,eas.ve.ready,44
23,eas.ve.loaded,44
3,eas.ve.first_change,25
19,eas.ve.save_intent,21
8,eas.ve.abort,21
14,eas.ve.save_success,20
22,eas.ve.save_attempt,20
4,eas.mf.init,6
10,eas.mf.first_change,6


The Visual Editor interface includes VE on mobile and desktop, as well as discussion tools events. In mediawiki_edit_attempt these are split into 'mf' for mobile frontend, 've' for desktop and 'dt' for discussion tools based on the mediawiki extension. 

All Visual Editor events are being logged successfully

### 3. Discussion Tools events

In [18]:
dt_eas_query=wmf.spark.run('''
SELECT 
  DISTINCT event.action AS action, 
  count(1) AS dt_events
  FROM event.editattemptstep
  WHERE event.editor_interface = 'visualeditor'
  AND event.integration='discussiontools'
  AND year = 2023
  AND month= 6
  AND wiki = 'testwiki' 
  GROUP BY action
''')

In [19]:
dt_eas_query.sort_values(by=['dt_events','action'], ascending=False)

,action,dt_events
7,ready,5
5,loaded,5
6,init,5
3,saveSuccess,2
2,saveIntent,2
0,saveAttempt,2
4,firstChange,2
1,abort,2


In [20]:
dt_mea_query= wmf.spark.run('''
SELECT 
  DISTINCT name, 
  count(1) AS dt_events
  FROM event.mediawiki_edit_attempt
  WHERE custom_data["editor_interface"].value = "visualeditor"
  AND name like 'eas.dt%'
  AND year = 2023
  AND month= 6
  AND mediawiki.database = "testwiki"
  GROUP BY name
''')

In [21]:
dt_mea_query.sort_values(by=['dt_events','name'], ascending=False)

,name,dt_events
7,eas.dt.ready,5
0,eas.dt.loaded,5
2,eas.dt.init,5
4,eas.dt.save_success,2
6,eas.dt.save_intent,2
1,eas.dt.save_attempt,2
5,eas.dt.first_change,2
3,eas.dt.abort,2


Discussion Tools events are logged successfully 

### 4. Mobile events

In [22]:
mf_eas_query=wmf.spark.run('''
SELECT 
  DISTINCT event.action AS action, 
  count(1) AS mobile_events
  FROM event.editattemptstep
  WHERE event.platform = 'phone'
  AND year = 2023
  AND month= 6
  AND wiki = 'testwiki' 
  GROUP BY action
''')

In [23]:
mf_mea_query= wmf.spark.run('''
SELECT 
  DISTINCT name, 
  count(1) AS mobile_events
  FROM event.mediawiki_edit_attempt
  WHERE agent.client_platform_family='mobile_browser'
  AND year = 2023
  AND month= 6
  AND mediawiki.database = "testwiki"
  GROUP BY name
''')


In [24]:
mf_mea_query.sort_values(by=['mobile_events','name'], ascending=False)

,name,mobile_events
1,eas.mf.ready,19
8,eas.mf.loaded,19
2,eas.mf.init,19
5,eas.mf.first_change,13
7,eas.mf.abort,8
3,eas.mf.save_intent,7
6,eas.mf.save_attempt,7
4,eas.mf.save_success,6
0,eas.mf.save_failure,1


In [25]:
mf_eas_query.sort_values(by=['mobile_events','action'], ascending=False)

,action,mobile_events
7,init,19
8,ready,18
6,loaded,18
5,firstChange,13
1,abort,8
2,saveIntent,7
0,saveAttempt,7
4,saveSuccess,6
3,saveFailure,1


In [26]:
mf_eas_query.mobile_events.sum()

97

In [27]:
mf_mea_query.mobile_events.sum()

99

Mobile events on `mediawiki_edit_attempt` are slightly higher. This is due to wiki editor events that are logged twice - once from `wt` extension and once from `mf` extension. This is an issue on both instrument platforms and will need to be fixed.  

### 4.1 Mobile wiki editor events

In [31]:
mf_wt_eas_query=wmf.spark.run('''
SELECT 
  DISTINCT event.action AS action, 
  count(1) AS mobile_events
  FROM event.editattemptstep
  WHERE event.platform = 'phone'
  AND event.editor_interface='wikitext'
  AND year = 2023
  AND month= 6
  AND wiki = 'testwiki' 
  GROUP BY action
''')

In [32]:
mf_wt_mea_query= wmf.spark.run('''
SELECT 
  DISTINCT name, 
  count(1) AS mobile_events
  FROM event.mediawiki_edit_attempt
  WHERE agent.client_platform_family='mobile_browser'
  AND custom_data["editor_interface"].value = "wikitext"
  AND year = 2023
  AND month= 6
  AND mediawiki.database = "testwiki"
  GROUP BY name
''')


In [33]:
mf_wt_mea_query.sort_values(by=['mobile_events','name'], ascending=False)

,name,mobile_events
1,eas.mf.ready,14
8,eas.mf.loaded,14
2,eas.mf.init,13
5,eas.mf.first_change,7
7,eas.mf.abort,6
3,eas.mf.save_intent,3
6,eas.mf.save_attempt,3
4,eas.mf.save_success,2
0,eas.mf.save_failure,1


In [34]:
mf_wt_eas_query.sort_values(by=['mobile_events','action'], ascending=False)

,action,mobile_events
8,ready,13
6,loaded,13
7,init,13
5,firstChange,7
1,abort,6
2,saveIntent,3
0,saveAttempt,3
4,saveSuccess,2
3,saveFailure,1


In [35]:
mf_wt_mea_query.mobile_events.sum()

63

In [36]:
mf_wt_eas_query.mobile_events.sum()

61

We can see that on Mobile, wikitext editor events are slightly higher on Metrics platform table `mediawiki_edit_attempt` than on legacy `editattemptstep`.   

Unlike the previous round of QA where we saw the same Mobile events being logged twice using `wt` and `mf` mediawiki extensions, this time we see them being logged only using the 'mf' extension. However Metrics platform is logging an additional event with a different timestamp (and sometimes a different timing value) for ready and loaded actions in a small proportion of editing sessions. 
We are not entirely sure if Metrics Platform is recording an extra event that shouldnt be logged or if legacy is not recording an event. In either case, if one editing session is programmed to have only have one ready or loaded event then this is a discrepancy to that design.

#### Further investigation into this 

In [64]:
mf_wt_mea_query1= wmf.spark.run('''
SELECT 
  dt, name, month, day, hour, 
  custom_data.editing_session_id, 
  custom_data.ready_timing, 
  custom_data.editor_interface,
  meta.request_id,
  performer.id
  FROM event.mediawiki_edit_attempt
  WHERE agent.client_platform_family='mobile_browser'
  AND custom_data["editor_interface"].value = "wikitext"
  AND year = 2023
  AND month= 6
  AND day=4
  AND hour=2
  AND mediawiki.database = "testwiki"
  AND name='eas.mf.ready'
  ORDER BY dt
''')


In [65]:
mf_wt_mea_query1

,dt,name,month,day,hour,editing_session_id,ready_timing,editor_interface,request_id,id
0,2023-06-04T02:05:14.959Z,eas.mf.ready,6,4,2,"(string, 147542f4c8ffd4302236e3b4ddc65220)","(number, 3856)","(string, wikitext)",aa57564e-faa8-47a3-ad3c-c297a4411569,0
1,2023-06-04T02:05:15.645Z,eas.mf.ready,6,4,2,"(string, 147542f4c8ffd4302236e3b4ddc65220)","(number, 700)","(string, wikitext)",988640c3-504e-4d4e-be2d-c726688d70ed,0


In [66]:
mf_wt_eas_query1=wmf.spark.run('''
SELECT 
  dt, event.action, month, day, hour, 
  event.editing_session_id, 
  event.ready_timing,
  event.user_id
  FROM event.editattemptstep
  WHERE event.platform = 'phone'
  AND event.editor_interface='wikitext'
  AND year = 2023
  AND month= 6
  AND day=4
  AND hour=2
  AND wiki = 'testwiki' 
  AND event.action='ready'
  ORDER BY dt
''')

In [67]:
mf_wt_eas_query1

,dt,action,month,day,hour,editing_session_id,ready_timing,user_id
0,2023-06-04T02:05:23.339Z,ready,6,4,2,147542f4c8ffd4302236e3b4ddc65220,700,0


In [101]:
# On mediawiki_edit_attempt
investigate_mf_wt_mea_query= wmf.spark.run('''
SELECT 
  dt, name, month, day, hour, 
  custom_data.editing_session_id, 
  custom_data.ready_timing, 
  custom_data.editor_interface,
  agent.client_platform_family,
  meta.request_id,
  performer.id
  FROM event.mediawiki_edit_attempt
  WHERE custom_data["editing_session_id"].value = "147542f4c8ffd4302236e3b4ddc65220"
  AND year = 2023
  AND month= 6
  ORDER BY dt
''')


In [102]:
investigate_mf_wt_mea_query

,dt,name,month,day,hour,editing_session_id,ready_timing,editor_interface,client_platform_family,request_id,id
0,2023-06-04T02:05:14.945Z,eas.mf.init,6,4,2,"(string, 147542f4c8ffd4302236e3b4ddc65220)",None,"(string, wikitext)",mobile_browser,14ad5837-2578-4446-81d4-5e93ab824065,0
1,2023-06-04T02:05:14.959Z,eas.mf.ready,6,4,2,"(string, 147542f4c8ffd4302236e3b4ddc65220)","(number, 3856)","(string, wikitext)",mobile_browser,aa57564e-faa8-47a3-ad3c-c297a4411569,0
2,2023-06-04T02:05:14.978Z,eas.mf.loaded,6,4,2,"(string, 147542f4c8ffd4302236e3b4ddc65220)",None,"(string, wikitext)",mobile_browser,83fc7f1b-ad28-430d-815d-1c1378b3ba2d,0
3,2023-06-04T02:05:15.645Z,eas.mf.loaded,6,4,2,"(string, 147542f4c8ffd4302236e3b4ddc65220)",None,"(string, wikitext)",mobile_browser,07b9974f-3568-47d7-963b-36f4d1b02b84,0
4,2023-06-04T02:05:15.645Z,eas.mf.ready,6,4,2,"(string, 147542f4c8ffd4302236e3b4ddc65220)","(number, 700)","(string, wikitext)",mobile_browser,988640c3-504e-4d4e-be2d-c726688d70ed,0


In [99]:
# On editattemptstep

investigate_mf_wt_eas_query=wmf.spark.run('''
SELECT 
  dt, event.action, month, day, hour, 
  event.editing_session_id, 
  event.ready_timing,
  event.platform,
  event.editor_interface,
  event.user_id
  FROM event.editattemptstep
  WHERE event.editing_session_id = "147542f4c8ffd4302236e3b4ddc65220"
  AND year = 2023
  AND month= 6
  ORDER BY dt
''')

In [100]:
investigate_mf_wt_eas_query

,dt,action,month,day,hour,editing_session_id,ready_timing,platform,editor_interface,user_id
0,2023-06-04T02:05:12Z,init,6,4,2,147542f4c8ffd4302236e3b4ddc65220,NaN,desktop,wikitext,0
1,2023-06-04T02:05:23.337Z,init,6,4,2,147542f4c8ffd4302236e3b4ddc65220,NaN,phone,wikitext,0
2,2023-06-04T02:05:23.338Z,ready,6,4,2,147542f4c8ffd4302236e3b4ddc65220,3856.0,desktop,wikitext,0
3,2023-06-04T02:05:23.338Z,loaded,6,4,2,147542f4c8ffd4302236e3b4ddc65220,NaN,desktop,wikitext,0
4,2023-06-04T02:05:23.339Z,ready,6,4,2,147542f4c8ffd4302236e3b4ddc65220,700.0,phone,wikitext,0
5,2023-06-04T02:05:23.342Z,loaded,6,4,2,147542f4c8ffd4302236e3b4ddc65220,NaN,phone,wikitext,0


**Conclusion:** A small proportion of mobile wikitext editing sessions log an additional (ready and loaded) event. This behavior is different than legacy editattemptstep and should be kept in mind during the implementation of Metrics platform schemas. 

Update: From Summary above, After discussing with @DLynch and @cjming, @DLynch made https://gerrit.wikimedia.org/r/c/mediawiki/extensions/WikiEditor/+/928926 to reapply the original patch which was merged and deployed Jun 15, 2023. As of Jun 20, we have not retested to see if the revert was successful in removing duplicate events.

### 4.2 Mobile VE events

In [68]:
mf_ve_eas_query=wmf.spark.run('''
SELECT 
  DISTINCT event.action AS action, 
  count(1) AS mobile_ve_events
  FROM event.editattemptstep
  WHERE event.platform = 'phone'
  AND event.editor_interface='visualeditor'
  AND event.integration='page' -- restricting to non discussion tools events only
  AND year = 2023
  AND month= 6
  AND wiki = 'testwiki' 
  GROUP BY action
''')

In [69]:
mf_ve_eas_query.sort_values(by=['mobile_ve_events','action'], ascending=False)

,action,mobile_ve_events
6,init,6
4,firstChange,6
7,ready,5
5,loaded,5
3,saveSuccess,4
2,saveIntent,4
0,saveAttempt,4
1,abort,2


In [70]:
mf_ve_mea_query= wmf.spark.run('''
SELECT 
  DISTINCT name, 
  count(1) AS mobile_ve_events
  FROM event.mediawiki_edit_attempt
  WHERE agent.client_platform_family='mobile_browser'
  AND custom_data["editor_interface"].value = "visualeditor"
  --AND name like 'eas.ve%'
  AND year = 2023
  AND month= 6
  AND mediawiki.database = "testwiki"
  GROUP BY name
''')


In [71]:
mf_ve_mea_query.sort_values(by=['mobile_ve_events','name'], ascending=False)

,name,mobile_ve_events
1,eas.mf.init,6
4,eas.mf.first_change,6
0,eas.mf.ready,5
7,eas.mf.loaded,5
3,eas.mf.save_success,4
2,eas.mf.save_intent,4
5,eas.mf.save_attempt,4
6,eas.mf.abort,2


In [72]:
mf_ve_eas_query.mobile_ve_events.sum()

36

In [73]:
mf_ve_mea_query.mobile_ve_events.sum()

36

On Mobile, Visual Editor events between the two tables are consistent (this comparison does not include discussion tool events)

### 5. Desktop events

In [74]:
desktop_eas_query=wmf.spark.run('''
SELECT 
  DISTINCT event.action AS action, 
  count(1) AS desktop_events
  FROM event.editattemptstep
  WHERE event.platform = 'desktop'
  AND year = 2023
  AND month= 6
  AND wiki = 'testwiki' 
  GROUP BY action
''')

In [75]:
desktop_eas_query.sort_values(by=['desktop_events','action'], ascending=False)

,action,desktop_events
7,ready,165
5,loaded,164
4,firstChange,91
6,init,78
1,abort,57
2,saveIntent,43
3,saveSuccess,39
0,saveAttempt,39


In [76]:
desktop_mea_query= wmf.spark.run('''
SELECT 
  DISTINCT name, 
  count(1) AS desktop_events
  FROM event.mediawiki_edit_attempt
  WHERE agent.client_platform_family='desktop_browser'
  AND year = 2023
  AND month= 6
  AND mediawiki.database = "testwiki"
  GROUP BY name
''')


In [77]:
desktop_mea_query.sort_values(by=['desktop_events','name'], ascending=False)

,name,desktop_events
3,eas.wt.ready,93
0,eas.wt.loaded,92
14,eas.ve.init,56
17,eas.ve.ready,52
19,eas.ve.loaded,52
5,eas.wt.first_change,43
6,eas.wt.abort,33
4,eas.ve.first_change,31
16,eas.ve.save_intent,26
8,eas.ve.abort,23


In [78]:
desktop_mea_query.desktop_events.sum()

680

In [79]:
desktop_eas_query.desktop_events.sum()

676

Once again, similar to Mobile, the number of Desktop events on mediawiki_edit_attempt is slightly higher than editattempstep. Lets investigate further..  

### 5.1 Desktop VE events

In [80]:
desktop_ve_eas_query=wmf.spark.run('''
SELECT 
  DISTINCT event.action AS action, 
  count(1) AS desktop_ve_events
  FROM event.editattemptstep
  WHERE event.platform = 'desktop'
  AND event.editor_interface='visualeditor'
  AND event.integration='page' -- restricting to non discussion tools events only
  AND year = 2023
  AND month= 6
  AND wiki = 'testwiki' 
  GROUP BY action
''')

In [81]:
desktop_ve_eas_query.sort_values(by=['desktop_ve_events','action'], ascending=False)

,action,desktop_ve_events
6,init,49
7,ready,45
5,loaded,45
4,firstChange,26
1,abort,22
2,saveIntent,21
3,saveSuccess,20
0,saveAttempt,20


In [82]:
desktop_ve_eas_query.desktop_ve_events.sum()

248

In [83]:
desktop_ve_mea_query= wmf.spark.run('''
SELECT 
  DISTINCT name, 
  count(1) AS desktop_ve_events
  FROM event.mediawiki_edit_attempt
  WHERE agent.client_platform_family='desktop_browser'
  AND custom_data["editor_interface"].value = "visualeditor"
  AND name like 'eas.ve%'
  AND year = 2023
  AND month= 6
  AND mediawiki.database = "testwiki"
  GROUP BY name
''')


In [84]:
desktop_ve_mea_query.sort_values(by=['desktop_ve_events','name'], ascending=False)

,name,desktop_ve_events
3,eas.ve.init,49
5,eas.ve.ready,45
7,eas.ve.loaded,45
0,eas.ve.first_change,26
1,eas.ve.abort,22
4,eas.ve.save_intent,21
2,eas.ve.save_success,20
6,eas.ve.save_attempt,20


In [85]:
desktop_ve_mea_query.desktop_ve_events.sum()

248

Visual Editor events on desktop are the same in both instruments

### 5.2 Desktop Wikitext events

In [87]:
desktop_wt_eas_query=wmf.spark.run('''
SELECT 
  DISTINCT event.action AS action, 
  count(1) AS desktop_events
  FROM event.editattemptstep
  WHERE event.platform = 'desktop'
  AND event.editor_interface='wikitext'
  AND year = 2023
  AND month= 6
  AND wiki = 'testwiki' 
  GROUP BY action
''')

In [88]:
desktop_wt_eas_query.sort_values(by=['desktop_events','action'], ascending=False)

,action,desktop_events
3,ready,92
2,loaded,91
1,firstChange,43
0,abort,31


In [89]:
desktop_wt_eas_query.desktop_events.sum()

257

In [90]:
desktop_wt_mea_query= wmf.spark.run('''
SELECT 
  DISTINCT name, 
  count(1) AS desktop_events
  FROM event.mediawiki_edit_attempt
  WHERE agent.client_platform_family='desktop_browser'
  AND custom_data["editor_interface"].value = "wikitext"
  AND year = 2023
  AND month= 6
  AND mediawiki.database = "testwiki"
  GROUP BY name
''')


In [91]:
desktop_wt_mea_query.sort_values(by=['desktop_events','name'], ascending=False)

,name,desktop_events
1,eas.wt.ready,93
0,eas.wt.loaded,92
2,eas.wt.first_change,43
3,eas.wt.abort,33


In [92]:
desktop_wt_mea_query.desktop_events.sum()

261

This observation is similar to the one for mobile, where the number of wikitext events is slightly higher in mediawiki_edit_attempt.
On Desktop we see that a small proportion of wikitext editing sessions log an additional ready, loaded and abort event. This is different than legacy editattemptstep, and should be kept in mind during the implementation of Metrics Platform. 